# Introduction

Most code is interpreted from: https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#

First, I prepare the necessary environment as shown in Riccardo's Demo file but adapt it to my project's paths:

In [1]:
import pyspark
import sys

In [2]:
src_gs = 'gs://stackoverflow_data_nicolas'

In [3]:
spark

In [4]:
SparkSession

pyspark.sql.session.SparkSession

Now import the post_answer and post_question datasets from the bucket:

In [5]:
SOanswers_df = spark.read.option('header', True).csv(path = src_gs + '/post_answers*')

In [6]:
SOquestions_df = spark.read.option('header', True).csv(path = src_gs + '/post_questions*')

In [7]:
import pyspark.sql.functions as f

# Question 1: - What percentage of questions have been answered over the years?

First, I explore SOanswers.df and SOquestions.df:

In [8]:
SOanswers_df.dtypes

[('id', 'string'),
 ('title', 'string'),
 ('body', 'string'),
 ('accepted_answer_id', 'string'),
 ('answer_count', 'string'),
 ('comment_count', 'string'),
 ('community_owned_date', 'string'),
 ('creation_date', 'string'),
 ('favorite_count', 'string'),
 ('last_activity_date', 'string'),
 ('last_edit_date', 'string'),
 ('last_editor_display_name', 'string'),
 ('last_editor_user_id', 'string'),
 ('owner_display_name', 'string'),
 ('owner_user_id', 'string'),
 ('parent_id', 'string'),
 ('post_type_id', 'string'),
 ('score', 'string'),
 ('tags', 'string'),
 ('view_count', 'string')]

In [9]:
SOquestions_df.dtypes

[('id', 'string'),
 ('title', 'string'),
 ('body', 'string'),
 ('accepted_answer_id', 'string'),
 ('answer_count', 'string'),
 ('comment_count', 'string'),
 ('community_owned_date', 'string'),
 ('creation_date', 'string'),
 ('favorite_count', 'string'),
 ('last_activity_date', 'string'),
 ('last_edit_date', 'string'),
 ('last_editor_display_name', 'string'),
 ('last_editor_user_id', 'string'),
 ('owner_display_name', 'string'),
 ('owner_user_id', 'string'),
 ('parent_id', 'string'),
 ('post_type_id', 'string'),
 ('score', 'string'),
 ('tags', 'string'),
 ('view_count', 'string')]

Considering that SOanswers and SOquestions have a column called answer_count, I create a new df with this column, the ID column and creation_date which I reformat from string to date: (https://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.functions.to_date)

In [10]:
Q1_answers = SOanswers_df.select(f.col('id'),f.year(f.to_date('creation_date')).alias('year'),f.col('answer_count'))
Q1_answers

DataFrame[id: string, year: int, answer_count: string]

In [11]:
Q1_questions = SOquestions_df.select(f.col('id'),f.year(f.to_date('creation_date')).alias('year'),f.col('answer_count'))
Q1_questions

DataFrame[id: string, year: int, answer_count: string]


Now, I group the dates into one year respectively, aggregate the answer_count and create a new column which all answers excluding any null values, hence excluding where no answer was given:

In [12]:
Q1_answers_new = Q1_answers.groupby(['year']).agg(f.count('answer_count').alias('number of answers'), f.count((f.when((f.col("answer_count")!='0')&(f.col("answer_count")!='null'),True))).alias('excluding no answers'))
Q1_answers_new


DataFrame[year: int, number of answers: bigint, excluding no answers: bigint]

In [ ]:
Q1_questions_new = Q1_questions.groupby(['year']).agg(f.count('answer_count').alias('number of answers'), f.count((f.when((f.col("answer_count")!='0')&(f.col("answer_count")!='null'),True))).alias('excluding no answers'))
Q1_questions_new

DataFrame[year: int, number of answers: bigint, excluding no answers: bigint]

Now, I create a new column with the percentage of answered questions whereby I divide total answers by 'excluding no answers' * 100. This answers Q1 considering the SOanswers datasets. To exclude outliers, the filter() function is used showing only entries with more than 100 results. 

In [ ]:
Q1_answers_final = Q1_answers_new.withColumn('percentage answered questions', (f.col('excluding no answers')/f.col('number of answers'))*100).filter(f.col('number of answers')>100).sort(f.col('year').desc()).show(10)

+----+-----------------+--------------------+-----------------------------+
|year|number of answers|excluding no answers|percentage answered questions|
+----+-----------------+--------------------+-----------------------------+
|2019|             2466|                2466|                        100.0|
|2018|             3900|                3875|            99.35897435897436|
|2017|             3782|                3782|                        100.0|
|2016|             4224|                4224|                        100.0|
|2015|             4601|                4601|                        100.0|
|2014|             5167|                5167|                        100.0|
|2013|             5299|                5299|                        100.0|
|2012|             4661|                4660|            99.97854537652864|
|2011|             4732|                4732|                        100.0|
|2010|             8593|                8593|                        100.0|
+----+------

Now, I create a new column with the percentage of answered questions whereby I divide total answers by 'excluding no answers' * 100. This answers Q1 considering the SOquestions datasets. To exclude outliers, the filter() function is used showing only entries with more than 100 results. 

In [ ]:
Q1_questions_final = Q1_questions_new.withColumn('percentage answered questions', (f.col('excluding no answers')/f.col('number of answers'))*100).filter(f.col('number of answers')>100).sort(f.col('year').desc()).show(10)

+----+-----------------+--------------------+-----------------------------+
|year|number of answers|excluding no answers|percentage answered questions|
+----+-----------------+--------------------+-----------------------------+
|2019|           111746|               70289|            62.90068548314929|
|2018|           153392|              116049|            75.65518410347345|
|2017|           165595|              137416|            82.98318185935565|
|2016|           178489|              151488|            84.87245712620945|
|2015|           184274|              159957|            86.80388985966549|
|2014|           180330|              160012|             88.7328786114346|
|2013|           185347|              170456|            91.96588021386913|
|2012|           172987|              164344|            95.00367079607138|
|2011|           150224|              146571|             97.5682980083076|
|2010|           104107|              103206|            99.13454426695611|
+----+------